### Exemplo de robô simples

Abaixo, um exemplo de loop de robô, que checa os preços a cada minutos e:
- Se a expectativa é de subida (+0,02%) e tem dinheiro em caixa, compra o equivalente a USD 10
- Se a expectativa é de queda (-0,02%) e tem a moeda em caixa, vende o equivalente a USD 10

In [1]:
import pickle
import time
from simple_robot import feature_eng, api_get, api_post, get_result, compute_quantity, how_much_i_have
import numpy as np
import pandas as pd

# Carregando modelo criado no notebook anterior
model = pickle.load(open('model_dummy.pickle', 'rb'))
token = 'token_dummy_001'
ticker = 'DOGEUSDT'
valor_compra_venda = 10

# Loop que checa o preço a cada minuto e toma as decisões de compra e venda.
while True:
    # Pegando o OHLC dos últimos 500 minutos
    df = api_post('cripto_quotation', {'token': token, 'ticker': ticker})
    
    # Realizando a engenharia de features
    df = feature_eng(df)
    
    # Isolando a linha mais recente
    df_last = df.iloc[[np.argmax(df['time'])]]
    
    # Calculando tendência, baseada no modelo linear criado
    tendencia = model.predict(df_last).iloc[0]
    
    # A quantidade de cripto que será comprada/ vendida depende do valor_compra_venda e da cotação atual
    qtdade = compute_quantity(coin_value = df_last['close'], invest_value = valor_compra_venda, significant_digits = 2)
    
    # Print do datetime atual
    print('-------------------')
    now = pd.to_datetime('now')
    print(f'{now}')
    
    if tendencia > 0.02:
        # Modelo detectou uma tendência positiva
        print(f"Tendência positiva de {str(tendencia)}")
        
        # Verifica quanto dinheiro tem em caixa
        qtdade_money = how_much_i_have('money', token)
        
        if qtdade_money>0:
            # Se tem dinheiro, tenta comprar o equivalente a qtdade ou o máximo que o dinheiro permitir
            max_qtdade = compute_quantity(coin_value = df_last['close'], invest_value = qtdade_money, significant_digits = 2)
            qtdade = min(qtdade, max_qtdade)
            
            # Realizando a compra
            print(f'Comprando {str(qtdade)} {ticker}')
            api_post('buy', payload = {'token': token, 'ticker': ticker, 'quantity': qtdade})
        
    elif tendencia < -0.02:
        # Modelo detectou uma tendência negativa
        print(f"Tendência negativa de {str(tendencia)}")
        
        # Verifica quanto tem da moeda em caixa
        qtdade_coin = how_much_i_have(ticker, token)
        
        if qtdade_coin>0:
            # Se tenho a moeda, vou vender!
            qtdade = min(qtdade_coin, qtdade)
            print(f'Vendendo {str(qtdade)} {ticker}')
            api_post('sell', payload = {'token': token, 'ticker': ticker, 'quantity':qtdade})
    else:
        # Não faz nenhuma ação, espera próximo loop
        print(f"Tendência neutra de {str(tendencia)}. Nenhuma ação realizada")
    
    # Print do status após cada iteração
    print(api_post('status', payload = {'token': token}))
    time.sleep(60)

-------------------
2021-08-08 21:08:48.227789
Tendência negativa de -0.0345967998723818
Vendendo 41.0 DOGEUSDT
     ticker      quantity
0   BTCUSDT  5.800000e+00
1  DOGEUSDT  4.959000e+03
2     money  1.008540e+07
3     total  1.034118e+07


KeyboardInterrupt: 